# 1. Crawler

## 1.0. Related example

This code shows `wget`-like tool written in python. Run it from console (`python wget.py`), make it work. Check the code, reuse, and modify for your needs.

In [ ]:
import argparse
import os
import re
import requests


def wget(url, filename):
    # allow redirects - in case file is relocated
    resp = requests.get(url, allow_redirects=True)
    # this can also be 2xx, but for simplicity now we stick to 200
    # you can also check for `resp.ok`
    if resp.status_code != 200:
        print(resp.status_code, resp.reason, 'for', url)
        return
    
    # just to be cool and print something
    print(*[f"{key}: {value}" for key, value in resp.headers.items()], sep='\n')
    print()
    
    # try to extract filename from url
    if filename is None:
        # start with http*, ends if ? or # appears (or none of)
        m = re.search("^http.*/([^/\?#]*)[\?#]?", url)
        filename = m.group(1)
        if not filename:
            raise NameError(f"Filename neither given, nor found for {url}")

    # what will you do in case 2 websites store file with the same name?
    if os.path.exists(filename):
        raise OSError(f"File {filename} already exists")
    
    with open(filename, 'wb') as f:
        f.write(resp.content)
        print(f"File saved as {filename}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='download file.')
    parser.add_argument("-O", type=str, default=None, dest='filename', help="output file name. Default -- taken from resource")
    parser.add_argument("url", type=str, default=None, help="Provide URL here")
    args = parser.parse_args()
    wget(args.url, args.filename)

### 1.0.1. How to parse a page?

If you build a crawler, you might follow one of the approaches:
1. search for URLs in the page, assuming this is just a text.
2. search for URLs in the places where URLs should appear: `<a href=..`, `<img src=...`, `<iframe src=...` and so on.

To follow the first approach you can rely on some good regular expression. [Like this](https://stackoverflow.com/a/3809435).

To follow the second approach just read one of these: [short answer](https://stackoverflow.com/questions/1080411/retrieve-links-from-web-page-using-python-and-beautifulsoup) or [exhaustive explanation](https://hackersandslackers.com/scraping-urls-with-beautifulsoup/).

## 1.1. [15] Download and persist #
Please complete a code for `load()`, `download()` and `persist()` methods of `Document` class. What they do:
- for a given URL `download()` method downloads binary data and stores in `self.content`. It returns `True` for success, else `False`.
- `persist()` method saves `self.content` somewhere in file system. We do it to avoid multiple downloads (for caching in other words).
- `load()` method loads data from hard drive. Returns `True` for success.

Tests checks that your code somehow works.

**NB Passing the test doesn't mean you correctly completed the task.** These are **criteria, which have to be fullfilled**:
1. URL is a unique identifier (as it is a subset of URI). Thus, documents with different URLs should be stored in different files. Typical errors: documents from the same domain are overwritten to the same file, URLs with similar endings are downloaded to the same file, etc.
2. The document can be not only a text file, but also a binary. Pay attention that if you download `mp3` file, it still can be played. Hint: don't hurry to convert everything to text.

In [1]:
import requests
from urllib.parse import quote

class Document:
    
    def __init__(self, url):
        self.url = url
        
    def get(self):
        if not self.load():
            if not self.download():
                raise FileNotFoundError(self.url)
            else:
                self.persist()
    
    def download(self):
        #TODO download self.url content, store it in self.content and return True in case of success
        response = requests.get(self.url)
        if response.ok:
            self.content = response.content
            return True
        else:
            return False
    
    def persist(self):
        #TODO write document content to hard drive
        try:
            file = "from_" + self.url.split("/")[2].split(".")[0] + "_got_file_" + self.url.rsplit("/", 1)[1]
            with open(file, "wb") as f:
                f.write(self.content)
            return True
        except:
            return False
            
    def load(self):
        #TODO load content from hard drive, store it in self.content and return True in case of success
        try:
            file = "from_" + self.url.split("/")[2].split(".")[0] + "_got_file_" + self.url.rsplit("/", 1)[1]
            with open(file, "rb") as f:
                self.content = f.read()
            return True
        except:
            return False

+ ### 1.1.1. Tests ###

In [2]:
doc = Document('http://sprotasov.ru/data/iu.txt')

doc.get()
assert doc.content, "Document download failed"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document content error"

doc.get()
assert doc.load(), "Load should return true for saved document"
assert "Code snippets, demos and labs for the course" in str(doc.content), "Document load from disk error"

## 1.2. [10] Parse HTML
`BeautifulSoap` library is a de facto standard to parse XML and HTML documents in python. Use it to complete `parse()` method that extracts document contents. You should initialize:
1. `self.anchors` list of tuples `('text', 'url')` met in a document. Be aware, there exist relative links (e.g. `../content/pic.jpg`). Use `urllib.parse.urljoin()` to fix this issue.
2. `self.images` list of images met in a document. Again, links can be relative to current page.
3. `self.text` should keep plain text of the document without scripts, tags, comments and so on. You can refer to [this stackoverflow answer](https://stackoverflow.com/a/1983219) for details.

**NB All these 3 criteria must be fulfilled to get full point for the task.**

In [3]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse


class HtmlDocument(Document):
    
    def parse(self):
        #TODO extract plain text, images and links from the document
        # self.anchors = [("fake link text", "http://fake.url/")]
        # self.images = ["http://image.com/fake.jpg"]
        # self.text = "fake text and some other text"

        self.anchors = []
        self.images = []
        self.text = " "

        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, "html.parser")

        for a in soup.find_all("a", href=True):
            url = urllib.parse.urljoin(self.url, a.get("href"))
            text = a.text
            self.anchors.append((text, url))

        for img in soup.find_all("img", src=True):
            src = urllib.parse.urljoin(self.url, img.get("src"))
            self.images.append(src)

        self.text = soup.text
        # tag = soup.body
        # for string in tag.strings:
        #     self.text = self.text + string.strip()



### 1.2.1. Tests

In [4]:
doc = HtmlDocument("http://sprotasov.ru")
doc.get()
doc.parse()

assert "just few links" in doc.text, "Error parsing text"
assert "http://sprotasov.ru/images/gb.svg" in doc.images, "Error parsing images"
assert any(p[1] == "https://twitter.com/07C3" for p in doc.anchors), "Error parsing links"

## 1.3. [10] Document analysis ##
Complete the code for `HtmlDocumentTextData` class. Implement word and sentence splitting (use any method you can propose). 

**Criteria to succeed in the task**: 
1. Your `get_word_stats()` method should return `Counter` object.
2. Don't forget to lowercase your words for counting.
3. Sentences should be obtained from inside `<body>` tag only.

In [5]:
from collections import Counter
from string import punctuation

class HtmlDocumentTextData:
    
    def __init__(self, url):
        self.url = url
        self.doc = HtmlDocument(url)
        self.doc.get()
        self.doc.parse()
    
    def get_sentences(self):
        #TODO implement sentence parser
        page = requests.get(self.url)
        soup = BeautifulSoup(page.content, "html.parser")

        result = []
        tag = soup.body
        for string in tag.strings:
            result.append(string.strip())
        return result

    # def get_sentences(self):
    #     #TODO implement sentence parser
    #     result = []
    #
    #     page = requests.get(self.url)
    #     soup = BeautifulSoup(page.content, "html.parser")
    #     result.append(soup.text.replace('\n', ' ').replace('\r', '').replace('\t', ' '))
    #     return result

    def get_word_stats(self):
        #TODO return Counter object of the document, containing mapping {`word` -> count_in_doc}
        sentences = self.get_sentences()

        counter = Counter((x.rstrip(punctuation).lower() for y in sentences for x in y.split()))

        return counter

### 1.3.1. Tests ###

In [6]:
doc = HtmlDocumentTextData("https://innopolis.university/")

print(doc.get_word_stats().most_common(10))
assert [x for x in doc.get_word_stats().most_common(10) if x[0] == 'иннополис'], 'иннополис should be among most common'

[('и', 44), ('в', 22), ('иннополис', 19), ('с', 13), ('на', 12), ('университета', 11), ('университет', 10), ('центр', 10), ('для', 9), ('образование', 8)]


## 1.4. [15] Crawling ##

Method `crawl_generator()` is given starting url (`source`) and max depth of search. It should return a **generator** of `HtmlDocumentTextData` objects (return a document as soon as it is downloaded and parsed). You can benefit from `yield obj_name` python construction. Use `HtmlDocumentTextData.anchors` field to go deeper.

In [7]:
from queue import Queue

class Crawler:
    def __init__(self, urls=[]):
        self.visited_urls = []
        self.urls_to_visit = urls

    def download_url(self, url):
        return requests.get(url).text

    def add_url_to_visit(self, url):
        if url not in self.visited_urls and url not in self.urls_to_visit:
            self.urls_to_visit.append(url)

    def crawl(self, doc):
        # html = doc.doc.text
        for url in doc.doc.anchors:
            self.add_url_to_visit(url[1])

    def crawl_generator(self, source, depth=1):
        #TODO return real crawling results. Don't forget to process failures, 
        # exceptions, 3**, 4** codes
        depth_counter = 0
        self.urls_to_visit.append(source)
        while self.urls_to_visit and depth_counter + 1 < depth:
            url = self.urls_to_visit.pop(0)
            doc = HtmlDocumentTextData(url)
            yield doc
            try:
                self.crawl(doc)
                depth_counter+=1
            except Exception:
                print(f'Failed to crawl: {url}')
            finally:
                self.visited_urls.append(url)


        # for url in self.visited_urls:
        #     yield url

### 1.4.1. Tests ###

In [8]:
crawler = Crawler()
counter = Counter()

for c in crawler.crawl_generator("https://innopolis.university/en/", 2):
    print(c.doc.url)
    if c.doc.url[-4:] in ('.pdf', '.mp3', '.avi', '.mp4', '.txt'):
        print("Skipping", c.doc.url)
        continue
    counter.update(c.get_word_stats())
    print(len(counter), "distinct word(s) so far")
    
print("Done")

print(counter.most_common(20))
assert [x for x in counter.most_common(20) if x[0] == 'innopolis'], 'innopolis sould be among most common'

https://innopolis.university/en/
340 distinct word(s) so far
Done
[('the', 32), ('and', 31), ('of', 28), ('university', 18), ('in', 15), ('innopolis', 14), ('education', 14), ('research', 12), ('for', 11), ('it', 10), ('international', 10), ('to', 10), ('development', 7), ('a', 7), ('robotics', 6), ('russian', 6), ('will', 6), ('faculty', 5), ('field', 5), ('higher', 5)]
